In [ ]:
import matplotlib
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
from scipy.stats import norm
plots.style.use('fivethirtyeight')

# Lecture 27

## Central Limit Theorem and Simulating Sample Mean ##

## The normal curve ##

In [ ]:
# The normal curve
x = np.arange(-4, 4, 0.001)
y = norm.pdf(x, 0, 1)

plots.plot(x, y)
plots.title("The Standard Normal Curve")
plots.xticks(np.arange(-4, 5, 1));

### Data is often normal

In [ ]:
# get information on baby births
births = Table.read_table('baby.csv')
births

In [ ]:
# heights are normally distributed
births.hist('Maternal Height', bins = np.arange(56.5, 72.6, 1), density=True)

In [ ]:
# weights are also normally distributed
births.hist('Birth Weight')

In [ ]:
# get the average and sd of weights
weights = births.column('Birth Weight')
mean_weight = np.mean(weights)
sd_weight = np.std(weights)
mean_weight, sd_weight

In [ ]:
# calculate the values of mean +/- 2 * SD
(mean_weight - 2 * sd_weight, mean_weight + 2 * sd_weight)

In [ ]:
# calculate percentiles for the middle 95% of the data
percentile(2.5, weights), percentile(97.5, weights)

In [ ]:
# Note: not all the data are normally distributed!
births.drop('Maternal Smoker').hist(overlay = False)

## Central Limit Theorem and Simulating Sample Mean ##

In [ ]:
# get the data set of flight delays
united = Table.read_table('united_summer2015.csv')
united

In [ ]:
# visualize the data (which is not normal!)
united.hist('Delay', bins = np.arange(-20, 300, 10))

In [ ]:
# calculate the mean and sd of the delays
delays = united.column('Delay')
mean_delay = np.mean(delays)
sd_delay = np.std(delays)

mean_delay, sd_delay

In [ ]:
# Take random sample from population of size sample_size 
# Repeat to get empirical distribution of sample average

sample_size = 400

means_400 = make_array()

for i in np.arange(10000):
    sampled_flights = united.sample(sample_size)
    sample_mean = np.mean(sampled_flights.column('Delay'))
    means_400 = np.append(means_400, sample_mean)

In [ ]:
# visualize the distribution of sample means
Table().with_columns('Sample Mean', means_400).hist(bins = 20)

plots.title('Sample Size ' + str(sample_size))
plots.xlabel('Sample Average')
print('Population Average: ', mean_delay);

In [ ]:
# look at the mean and sd of our means_400 array
np.average(means_400), np.std(means_400)

In [ ]:
# look at the mean of the original data
mean_delay, sd_delay

In [ ]:
# look at the original SD divided by square root of n
sd_delay/np.sqrt(sample_size)

## Variability of the Sample Average ##

In [ ]:
# Let's examine a distribution of sample means, where each mean comes from a larger sample size

sample_size = 900

means_900 = make_array()

for i in np.arange(10000):
    sampled_flights = united.sample(sample_size)
    sample_mean = np.mean(sampled_flights.column('Delay'))
    means_900 = np.append(means_900, sample_mean)

In [ ]:
# compare the sampling distributions for sample sizes of n = 400, and n = 900
means_tbl = Table().with_columns(
    '400', means_400,
    '900', means_900
)

In [ ]:
means_tbl.hist(bins = np.arange(5, 31, 0.5))
plots.title('Distribution of Sample Average');

### Exploring the relationship between sample size (n) and the spread of the sampling distribution (SE)

In [ ]:
# a function that computes the sampling distribution of sample means for the United flight delays

def sample_means(sample_size):
    """Empirical distribution of random sample means"""
    
    repetitions = 10000
    means = make_array()

    for i in range(repetitions):
        sampled_flights = united.sample(sample_size)
        sample_mean = np.mean(sampled_flights.column('Delay'))
        means = np.append(means, sample_mean)

    sample_means = Table().with_column('Sample Means', means)
    
    # Display empirical histogram and print all relevant quantities
    sample_means.hist(bins=np.arange(5, 36, 1))
    plots.xlabel('Sample Means')
    plots.title('Sample Size ' + str(sample_size))
    print("Sample size: ", sample_size)
    print("Population mean:", np.mean(united.column('Delay')))
    print("Average of sample means: ", np.mean(means))
    print("Population SD:", np.std(united.column('Delay')))
    print("SD of sample means:", np.std(means))

In [ ]:
# run the function for a sample size of n = 100
sample_means(100)

In [ ]:
# run the function for a sample size of n = 400
sample_means(400)

In [ ]:
# run the function for a sample size of n = 625
sample_means(625)

In [ ]:
# compute the standard errors for different sample sizes
sd_delay, sd_delay / np.sqrt(make_array(100, 400, 625))

In [ ]:
# compare standard error estimates for a sequence of sample sizes
sample_sizes = np.arange(50, 401, 50)

sd_of_sample_means = make_array()

for n in sample_sizes:
    means = make_array()
    for i in np.arange(10000):
        means = np.append(means, np.mean(united.sample(n).column('Delay')))
    sd_of_sample_means = np.append(sd_of_sample_means, np.std(means))

In [ ]:
# display the results in a table
sd_comparison = Table().with_columns(
    'Sample Size n', sample_sizes,
    'SD of 10,000 Sample Means', sd_of_sample_means,
    'Population_SD/sqrt(n)', sd_delay/np.sqrt(sample_sizes)
)

In [ ]:
sd_comparison

In [ ]:
# visualize the comparison of theoretical SEs and empirical SEs
sd_comparison.scatter('Sample Size n')